In [1]:
# import widen_notebook
from mysetup import NotebookFinder
import sys

sys.meta_path.append(NotebookFinder())

from setup_transform import *

%matplotlib inline
cl_weight = sklearn.utils.compute_class_weight(
    class_weight="balanced", classes=[0, 1, 2], y=y
)
CLASS_WEIGHTS = {i: cl_weight[i] for i in range(3)}
def_cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=5, random_state=42)
lgr_params = dict(
    class_weight=CLASS_WEIGHTS,
    fit_intercept=False,
    multi_class="ovr",
    max_iter=2000000,
    random_state=42,
    n_jobs=24,
    #     penalty="elasticnet",
    cv=def_cv,
    scoring="f1_macro",
    solver="lbfgs",
    Cs=100,
    #     l1_ratios=np.linspace(0, 1, endpoint=False, num=100),
)
from sklearnex import unpatch_sklearn

import copy
import warnings

warnings.filterwarnings("ignore")
discrete = nominal + discrete_ordinal + discrete_binary
X_master = pd.concat(
    [raw_data.loc[:, raw_data_eval.columns], raw_data_eval], ignore_index=True, axis=0
)

importing Jupyter notebook from setup_transform.ipynb


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from pprint import pprint as pp

In [ ]:
ALPHA = 1.0e-10
clf_1 = CategoricalNB(alpha=ALPHA, fit_prior=True, min_categories=2)
clf_2 = ComplementNB(alpha=ALPHA, fit_prior=True, norm=False)
clf_3 = BernoulliNB(binarize=False, alpha=ALPHA, fit_prior=True)
clf_4 = MultinomialNB(alpha=ALPHA, fit_prior=True)
# clf1_ = make_pipeline(VarianceThreshold(0.0001),clf_1)
# clf2_ = make_pipeline(VarianceThreshold(0.0001),clf_2)
# clf3_  = make_pipeline(VarianceThreshold(0.0001),clf_3)
# clf4_  = make_pipeline(VarianceThreshold(0.0001),clf_4)


clfs = []
for clf in [clf_1, clf_2, clf_3, clf_4]:
    clfs.append(OneVsOneClassifier(clf, n_jobs=-1))
# clfs = [clf1_,clf2_,clf3_,clf4_]

ct = make_column_transformer(
    (OneHotEncoder(sparse=False), nominal + discrete_ordinal),
    ("passthrough", discrete_binary),
    sparse_threshold=0,
    n_jobs=-1,
)
ct.fit(X_raw)

X, y = (
    pd.DataFrame(ct.transform(X_raw), columns=cleanup_feature_names(ct)),
    raw_data.target,
)

other = X_raw.index.difference(fin_idx)

X_train, X_test, y_train, y_test = gen_train_test(
    X.loc[other, :], y.loc[other], test_size=0.3
)
# X.
# clfs = [clf_1, clf_2, clf_3, clf_4]
stc = StackingClassifier(
    estimators=[(f"clf_{i}", clfs[i]) for i in range(4)],
    final_estimator=LogisticRegressionCV(**lgr_params),
    n_jobs=24,
)

In [ ]:
res = []
with parallel_backend("loky"):
    for clf in stc.estimators_:
        #     sc = cross_validate(clf,X,y,cv=RepeatedStratifiedKFold(),n_jobs=-1,scoring='f1_macro',)
        #     res.append(sc)
        #     clf = clf_[-1]
        clf.partial_fit(X.loc[fin_idx, :], y.loc[fin_idx], classes=[0, 1, 2])
        #     clf.partial_fit(X[fin_idx,:],y.loc[fin_idx],classes=[0,1,2])
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=20)

        for train_index, test_index in cv.split(X_train, y_train):
            X_train_, X_test_ = X.loc[train_index, :], X.loc[test_index, :]
            y_train_, y_test_ = y.loc[train_index], y.loc[test_index]
            #         clf.partial_fit(X_test_,y_test_)
            clf.partial_fit(X_test_, y_test_, classes=[0, 1, 2])
            #         print(X_train.shape)

            clf.partial_fit(X_train_, y_train_, classes=[0, 1, 2])
    #         clf.partial_fit(X_train_,y_train_)
y_preds = [clf.predict(X_test) for clf in clfs]
y_pred_base = clf.fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, y_pred_base))
for y_pred in y_preds:
    print(classification_report(y_test, y_pred))
clfs = [clf_1, clf_2, clf_3, clf_4]
stc = StackingClassifier(
    estimators=[(f"clf_{i}", clfs[i]) for i in range(4)],
    final_estimator=LogisticRegressionCV(**lgr_params),
    n_jobs=24,
)
stc
X_transformed = stc.transform(X_test)
stc.final_estimator_.fit(X_test, y_test)
# y_pred = stc.fit(X_train,y_train).predict(X_test)
y_pred = stc.predict(X_test)
print(classification_report(y_test, y_pred))

```text
            precision    recall  f1-score   support

         0.0       0.76      0.60      0.67       345
         1.0       0.79      0.78      0.78       531
         2.0       0.43      0.59      0.50       199

    accuracy                           0.69      1075
   macro avg       0.66      0.66      0.65      1075
weighted avg       0.71      0.69      0.69      1075

              precision    recall  f1-score   support

         0.0       0.77      0.68      0.72       345
         1.0       0.84      0.79      0.82       531
         2.0       0.48      0.65      0.55       199

    accuracy                           0.73      1075
   macro avg       0.70      0.71      0.70      1075
weighted avg       0.75      0.73      0.74      1075

              precision    recall  f1-score   support

         0.0       0.79      0.66      0.72       345
         1.0       0.85      0.78      0.81       531
         2.0       0.45      0.67      0.54       199

    accuracy                           0.72      1075
   macro avg       0.69      0.70      0.69      1075
weighted avg       0.75      0.72      0.73      1075

              precision    recall  f1-score   support

         0.0       0.77      0.68      0.72       345
         1.0       0.84      0.79      0.82       531
         2.0       0.48      0.65      0.55       199

    accuracy                           0.73      1075
   macro avg       0.70      0.71      0.70      1075
weighted avg       0.75      0.73      0.74      1075

              precision    recall  f1-score   support

         0.0       0.78      0.68      0.73       345
         1.0       0.84      0.81      0.82       531
         2.0       0.48      0.64      0.55       199

    accuracy                           0.73      1075
   macro avg       0.70      0.71      0.70      1075
weighted avg       0.75      0.73      0.74      1075
```

In [ ]:
Ne = [
    "nominal__v_12",
    "nominal__v_18",
    "nominal__v_20",
    "nominal__v_25",
    "nominal__v_3",
    "nominal__v_32",
    "nominal__v_4",
]
df = X_master[Ne]
train = raw_data
evale = raw_data_eval
all_infreq_idx_train = None
all_infreq_idx_eval = None
perf = {}
N_dims = X_master[nominal].nunique().sum()
for dims in range(2, 30):
    dim_reduction = 0

    for c in Ne:
        #         print(":"*80)
        #     pp()
        vc = df[c].value_counts().sort_values(ascending=False)
        vc_ = vc[vc < dims]
        t_samples = vc_.sum()
        dim_reduction += vc_.shape[0]
        #         print(c,vc_.shape[0])
        vc_ = vc_.to_dict()
        v_t = train[c].value_counts().sort_values(ascending=False).to_dict()
        v_e = evale[c].value_counts().sort_values(ascending=False).to_dict()
        N_train = 0
        N_eval = 0
        for val in vc_:
            if val in v_t:
                N_train += v_t[val]
            if val in v_e:
                N_eval += v_e[val]
        #         print("Training Samples : ", N_train)
        #         print("Eval Samples : ", N_eval)
        #         print("Total Samples: ", t_samples)

        for val in vc_:
            if type(all_infreq_idx_train) == pd.core.indexes.numeric.Int64Index:
                all_infreq_idx_train = all_infreq_idx_train.union(
                    raw_data[raw_data[c] == val].index
                )
            else:
                all_infreq_idx_train = raw_data[raw_data[c] == val].index
            if type(all_infreq_idx_eval) == pd.core.indexes.numeric.Int64Index:
                all_infreq_idx_eval = all_infreq_idx_eval.union(
                    raw_data_eval[raw_data[c] == val].index
                )
            else:
                all_infreq_idx_eval = raw_data_eval[raw_data[c] == val].index
    #     print(":"*80)
    #     print(":"*80)
    try:
        v1 = (len(all_infreq_idx_train) / 3796) * 100
    except TypeError as e:
        v1 = 0

    try:
        v2 = (len(all_infreq_idx_eval) / 1628) * 100
    except TypeError as e:
        v2 = 0

    try:
        v3 = (dim_reduction / N_dims) * 100
    except TypeError as e:
        v3 = 0

    print("Total Training Samples:", v1, "%")
    print("Total Eval Samples:", v2, "%")
    print("Total Dim Reduction:", v3, "%")

    perf[dims] = {
        "Total Training Samples": v1,
        "Total Eval Samples": v2,
        "Total Dim Reduction": v3,
    }

In [ ]:
perf_df = pd.DataFrame(perf).transpose()
_ = perf_df.plot(xticks=np.arange(2, 30), figsize=(20, 9))

In [ ]:
perf_df.sort_values(by=['Total Dim Reduction'],ascending=False).

In [27]:
X_ohe.nunique().to_numpy()

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2

In [31]:
from tqdm import tqdm, trange


def train_nb_clf(clf, X, y):
    X_train, X_test, y_train, y_test = gen_train_test(X, y, test_size=0.2)
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=36851234)
    progd = []
    for train_index, test_index in rskf.split(X_train, y_train):
        #         print("TRAIN:", train_index, "TEST:", test_index)
        X_train_, X_test_ = X.loc[train_index, :], X.loc[test_index, :]
        y_train_, y_test_ = y.loc[train_index], y.loc[test_index]
        y_pred_ = clf.partial_fit(X_train_, y_train_, classes=[0, 1, 2])
        y_pred_ = clf.partial_fit(X_test_, y_test_, classes=[0, 1, 2])
    #         prog = f1_score(y_test,clf.predict(X_test),average='macro')
    #         progd.append(prog)
    clf.partial_fit(X_train, y_train)
    prog = f1_score(y_test, clf.predict(X_test), average="macro")
    progd.append(prog)
    print(np.array(progd).mean())
    #     fig,ax = plt.subplots(1,1,figsize=(7,5))
    #     _  = pd.Series(progd).plot(ax=ax,title=str(X.nunique().sum()))
    return progd


vc_all = {c: X_master[c].value_counts() for c in nominal}
n_neighbours = (
    list(np.arange(2, 10)) + list(np.arange(10, 50, 5)) + list(np.arange(50, 500, 25))
)
for c in nominal + discrete_ordinal + discrete_binary:
    raw_data[c] = raw_data[c].astype(np.uint16)
y = raw_data.target
ohe = make_column_transformer(
    (OneHotEncoder(sparse=False, dtype=np.int64), nominal),
    remainder="passthrough",
    sparse_threshold=0,
)
chi2_ohe = make_column_transformer(
    (OneHotEncoder(sparse=False, dtype=np.int64), nominal),
    ("passthrough", discrete_binary + discrete_ordinal),
    remainder="drop",
    sparse_threshold=0,
)
# chi2_ohe = make_column_transformer(('passthrough',nominal),remainder='drop',sparse_threshold=0)
def discrete_features_list(df):
    d_mask = []
    for c in df.columns:
        if c in discrete_ordinal + discrete_binary or "nominal" in c:
            d_mask.append(True)
        else:
            d_mask.append(False)
    return np.array(d_mask)


master_chi_X = {}
master_chi_R = {}


def remove_infreq(row, CUT):
    for c in nominal:
        if vc_all[c][row[c]] <= CUT:
            row[c] = max(list(vc_all[c].keys())) + 1
    return row


clf = CategoricalNB(
    alpha=1e-10, min_categories=X_ohe.nunique().to_numpy(), fit_prior=True
)
dif_trained_clfs = []
with parallel_backend("threading", n_jobs=24):
    for cut_val in range(3, 30):
        clf = OneVsRestClassifier(
            CategoricalNB(
                alpha=1e-10, min_categories=X_ohe.nunique().to_numpy(), fit_prior=False
            )
        )
        X_ = raw_data[raw_data_eval.columns].copy()
        X_ = X_.apply(remove_infreq, axis=1, CUT=cut_val)
        X_ohe = chi2_ohe.fit_transform(X_)
        X_ohe = pd.DataFrame(X_ohe, columns=cleanup_feature_names(chi2_ohe)).astype(
            "int"
        )
        d_f_x = discrete_features_list(X_ohe)
        clf = OneVsRestClassifier(
            CategoricalNB(
                alpha=1e-10, min_categories=X_ohe.nunique().to_numpy(), fit_prior=False
            )
        )
        fit_clf = train_nb_clf(clf, X_ohe, y)
        dif_trained_clfs.append(fit_clf)

r_ohe = chi2_ohe.fit_transform(raw_data[raw_data_eval.columns])
r_ohe = pd.DataFrame(r_ohe, columns=cleanup_feature_names(chi2_ohe))
d_f_r = discrete_features_list(r_ohe)
#     mic_R = {}
#     for i in tqdm(n_neighbours):
#         mic_R[i] = chi2(r_ohe,y)
#     for cut_val in trange(3,30):
clf = OneVsRestClassifier(
    CategoricalNB(
        alpha=1e-10, min_categories=r_ohe.nunique().to_numpy(), fit_prior=False
    )
)
master_clf_R = train_nb_clf(clf, r_ohe, y)
#         master_chi_R[cut_val] = chi2(r_ohe,y)
#         break

#     break
#                    X
#                   /
# CUT-->N_neighbours-
#                   \
#                    R

0.6628778516299143
0.6628778516299143
0.6639604351225585
0.6628778516299143
0.6628778516299143
0.6628778516299143
0.6628778516299143
0.6628778516299143
0.6648675280388835
0.6627297254206158
0.6585990515396233
0.6585990515396233
0.656620964321459
0.6545149281256452
0.6545050457435962
0.6535828389453561
0.6511245447008182
0.6525273112608986
0.6511245447008182
0.6514534794778675
0.6516168249553494
0.6502618843416483
0.6487293452487045
0.6486933419780622
0.6470045106408743
0.6459517127464773
0.6479642697711687
0.6662392446428608


In [18]:
dif_trained_clfs[0]

[0.4533015163571735,
 0.4799144562775736,
 0.5479351847338002,
 0.5268523200298126,
 0.5459953999939327,
 0.5152393744021359,
 0.486821629172036,
 0.5536721563951611,
 0.5282478189628449,
 0.5543883957504658,
 0.5377248324217212,
 0.5184996958093436,
 0.5169879189316409,
 0.5269572956730365,
 0.5387495984181067,
 0.5432321337329459,
 0.5167735996790391,
 0.5568086967331752,
 0.5135989532166675,
 0.5048815415012599,
 0.5136193228015323,
 0.5368060362064387,
 0.4936750551607651,
 0.5372978060659855,
 0.564204306786504,
 0.5457636500003606,
 0.5163914475032275,
 0.5290679326785201,
 0.5341961657198085,
 0.5270424638176714,
 0.5098954164144816,
 0.5292042866298704,
 0.5386671721349141,
 0.5308203765638465,
 0.5386962346165381,
 0.50839042427192,
 0.5388602881657762,
 0.5506368585169917,
 0.5194249503552978,
 0.5376554751940547,
 0.5183420880969547,
 0.5449995485273238,
 0.5477220448381227,
 0.5384791010015327,
 0.5051784404588533,
 0.5159399397726356,
 0.5550521148898432,
 0.53587021163815

In [2]:
def train_nb_clf(clf, X, y):
    X_train, X_test, y_train, y_test = gen_train_test(X, y, test_size=0.2)
    rskf = RepeatedStratifiedKFold(n_splits=20, n_repeats=20, random_state=36851234)
    progd = []
    for train_index, test_index in rskf.split(X_train, y_train):
        #         print("TRAIN:", train_index, "TEST:", test_index)
        X_train_, X_test_ = X[train_index], X[test_index]
        y_train_, y_test_ = y[train_index], y[test_index]
        y_pred_ = clf.partial_fit(X_train_, y_train_)
        prog = f1_score(y_test_, clf.predict(X_test_), average="macro")
        progd.append(prog)
    prog = f1_score(y_test, clf.predict(X_test), average="macro")
    progd.append(prog)
    _ = pd.Series(progd).plot()
    return clf

In [ ]:
print(X_ohe.shape)
X_ohe.var()
r_ohe.var()

In [ ]:
master_chi_X
chi_sig = []
chi_sig2 = []
for thr in master_chi_X:
    chi_val, p = master_chi_X[thr]
    chi_sig.append(len(p[p > 0.05]))
for thr in master_chi_R:
    chi_val, p = master_chi_R[thr]
    chi_sig2.append(len(p[p > 0.05]))
# _ = plt.plot(chi_sig)
# _ = plt.plot(chi_sig2)
chi_sig = np.array(chi_sig)
chi_sig2 = np.array(chi_sig2)
# print(chi_sig)
print(list(zip(list(np.arange(3, 30)), chi_sig - chi_sig2)))
print()

In [ ]:
joblib.dump([master_X, master_R], "../data/MIC")

In [ ]:
mdf_X = pd.DataFrame(master_X)
l, b = mdf_X.shape[0], mdf_X.shape[1]
mdf_X_var = mdf_X.copy()
mdf_X_std = mdf_X.copy()
for i in range(l):
    for j in range(b):
        mdf_X_var.iloc[i, j] = mdf_X_var.iloc[i, j].var()
        mdf_X_std.iloc[i, j] = mdf_X_std.iloc[i, j].std()

In [ ]:
mdf_X_var * 100000

In [ ]:
mdf_X * 1000
e = np.empty((253,))
X_stack_N = mdf_X.loc[:, 3].to_numpy()
for s_ in X_stack_N:
    #     print(len(s_))
    e = np.c_[e, s_]
#     print(s_.shape)

In [ ]:
X_stack_N = mdf_X.loc[:, 12].to_numpy()
for s_ in X_stack_N:
    #     e = np.c_[e,s_]
    print(s_.shape)

In [ ]:
# X_mic = np.zeros((27,35))
mdf.head()

In [ ]:
for t in master:
    for n in master[t]:
        n["X"].mean() - n["R"].mean()